# __Loading and Chunking Text Data__

# Langchain Document

Langchain object to store an ingested document.

Main parts:

## page_content
- one single String
- containing the complete text content of the document - or the chunk after chunking

## metadata
- a dictionary with
- - source (file path or URL)
- - page and chunk id (e.g. 'page: 42, chunk: 7)
- - timestamp (of creation/modification)
- - author
- - category ('technical', 'legal', etc)
- - language ('en', 'de', etc)

### Import the libraries

In [8]:
import os
from typing import List, Dict, Any
import pandas as pd

from langchain_core.documents import Document
from langchain_text_splitters.character import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter
)
from langchain_text_splitters.base import TokenTextSplitter
print("Imports successful")

Imports successful


# Create a simple document manually

In [10]:
doc = Document(
    page_content='''Lorem ipsum dolor sit amet consectetur adipiscing elit. 
    Quisque faucibus ex sapien vitae pellentesque sem placerat. In id cursus mi pretium tellus duis convallis. 
    Tempus leo eu aenean sed diam urna tempor. Pulvinar vivamus fringilla lacus nec metus bibendum egestas. 
    Iaculis massa nisl malesuada lacinia integer nunc posuere. Ut hendrerit semper vel class aptent taciti sociosqu. 
    Ad litora torquent per conubia nostra inceptos himenaeos.''',
    metadata={
        "source": "example.txt",
        "page": 1,
        "author": "Some Body",
        "date_created": "2024-06-01",
        "length": 100,
        "tags": ["test", "example"],
        "my_own_field": "my_own_value"
    }
)

print(f"Content: {doc.page_content}")
print(f"Metadata: {doc.metadata}")

Content: Lorem ipsum dolor sit amet consectetur adipiscing elit. 
    Quisque faucibus ex sapien vitae pellentesque sem placerat. In id cursus mi pretium tellus duis convallis. 
    Tempus leo eu aenean sed diam urna tempor. Pulvinar vivamus fringilla lacus nec metus bibendum egestas. 
    Iaculis massa nisl malesuada lacinia integer nunc posuere. Ut hendrerit semper vel class aptent taciti sociosqu. 
    Ad litora torquent per conubia nostra inceptos himenaeos.
Metadata: {'source': 'example.txt', 'page': 1, 'author': 'Some Body', 'date_created': '2024-06-01', 'length': 100, 'tags': ['test', 'example'], 'my_own_field': 'my_own_value'}


# Meatadata is used for
- filtering search results
- tracking document sources
- providing context in responses
- debugging and auditing

E.g. if the query is "Who wrote Pulvinar vivamus fringilla", the VDB can find that it was Some Body.

In [11]:
type(doc)

langchain_core.documents.base.Document

# Text File

## Create a simply text file

In [12]:
os.makedirs("data/text_files", exist_ok=True)

In [13]:
sample_texts = {
    "data/text_files/python.txt": '''Python is a widely-used high-level programming language celebrated for its clear syntax and ease of learning. 
    Developed by Guido van Rossum and first released in 1991, Python emphasizes readability and simplicity, 
    making it accessible to both beginners and seasoned developers. It supports multiple programming paradigms, 
    including procedural, object-oriented, and functional programming. Python's extensive standard library and vibrant 
    community contribute to its versatility, enabling it to be utilized across various domains such as web development, 
    data analysis, artificial intelligence, machine learning, automation, and scientific computing. Its adaptability and 
    continuous development ensure that Python remains a fundamental tool in the software development industry, fostering 
    innovation and productivity.''',

    "data/text_files/java.txt": '''Java is a widely-used, high-level programming language renowned for its portability, 
    security, and robustness. Developed by James Gosling and released by Sun Microsystems in 1995, Java's core philosophy 
    is "write once, run anywhere," meaning that code written in Java can run on any device or operating system that has a 
    Java Virtual Machine (JVM). This platform independence, combined with its object-oriented nature, makes Java a popular 
    choice for building large-scale enterprise applications, mobile apps (especially Android), web applications, and 
    embedded systems. Java's extensive libraries, strong community support, and emphasis on reliability have 
    contributed to its long-standing popularity and importance in the software development industry.''',
    
    "data/text_files/rust.txt": '''Rust is a modern systems programming language designed for speed, safety, and concurrency. 
    Developed by Mozilla and first released in 2010, Rust aims to provide the performance of low-level languages 
    like C and C++, while eliminating many of their common bugs and security vulnerabilities. It features a 
    unique ownership model that enforces memory safety without the need for a garbage collector, making it 
    ideal for developing high-performance, reliable software. Rust is increasingly popular for systems development, 
    WebAssembly, embedded systems, and performance-critical applications. Its emphasis on safety, combined with
    modern syntax and powerful tooling, has made Rust a favorite among developers seeking a safe yet efficient 
    programming environment.'''
}

In [14]:
for file_path, content in sample_texts.items():
    with open(file_path, 'w') as f:
        f.write(content)

# `TextLoader()` - Read a __single file__

For this we need a loader

In [17]:
from langchain_community.document_loaders.text import TextLoader

### Create a loader for a file:

In [19]:
loader = TextLoader("data/text_files/python.txt", encoding='utf-8')

### Load:

In [23]:
docs = loader.load()
print(type(docs))  
print(type(docs[0]))
print(len(docs))
print(docs[0].page_content[:200])
print()
print(docs[0].metadata)

<class 'list'>
<class 'langchain_core.documents.base.Document'>
1
Python is a widely-used high-level programming language celebrated for its clear syntax and ease of learning. 
    Developed by Guido van Rossum and first released in 1991, Python emphasizes readabili

{'source': 'data/text_files/python.txt'}


# Read __all__ text files from a directory - `DirectoryLoader()`

In [25]:
from langchain_community.document_loaders.directory import DirectoryLoader

In [27]:
dir_loader = DirectoryLoader(
    "data/text_files",                          # Directory containing text files
    glob="**/*.txt",                            # REGEX Pattern to match text files - recursive search for all .txt files
    loader_cls=TextLoader,                      # Loader class to use for each file
    loader_kwargs={"encoding": "utf-8"},
    show_progress=True                          # Show progress bar
)

docs = dir_loader.load()
print(f"Total documents loaded: {len(docs)}")  
print(f"First document content preview: {docs[0].page_content[:200]}")
print()
for i, doc in enumerate(docs):
    print(f"Document {i+1} metadata: {doc.metadata}")


100%|██████████| 3/3 [00:00<?, ?it/s]

Total documents loaded: 3
First document content preview: Python is a widely-used high-level programming language celebrated for its clear syntax and ease of learning. 
    Developed by Guido van Rossum and first released in 1991, Python emphasizes readabili

Document 1 metadata: {'source': 'data\\text_files\\python.txt'}
Document 2 metadata: {'source': 'data\\text_files\\java\\java.txt'}
Document 3 metadata: {'source': 'data\\text_files\\funny_langs\\the_funniest\\rust.txt'}


# Text Splitting Strategies

Every LLM model has limitation, how big context can digest.

## `CharacterTextSplitter()`

In [28]:
from langchain_text_splitters.character import CharacterTextSplitter

Get the text content of the 1st document

In [29]:
text = docs[0].page_content

We need to know, what is the __new line__ character, so have a look:

In [30]:
text

"Python is a widely-used high-level programming language celebrated for its clear syntax and ease of learning. \n    Developed by Guido van Rossum and first released in 1991, Python emphasizes readability and simplicity, \n    making it accessible to both beginners and seasoned developers. It supports multiple programming paradigms, \n    including procedural, object-oriented, and functional programming. Python's extensive standard library and vibrant \n    community contribute to its versatility, enabling it to be utilized across various domains such as web development, \n    data analysis, artificial intelligence, machine learning, automation, and scientific computing. Its adaptability and \n    continuous development ensure that Python remains a fundamental tool in the software development industry, fostering \n    innovation and productivity."

OK, it is simply `\n`, so make the splitter

In [31]:
char_splitter = CharacterTextSplitter(
    separator="\n",        # Split at new lines 
    chunk_size=100,       # Each chunk will be at most 100 characters
    chunk_overlap=20,     # Overlap between chunks
    length_function=len    # Function to calculate length (default is len
)

Split with it

In [32]:
char_chunks = char_splitter.split_text(text)

Created a chunk of size 110, which is longer than the specified 100
Created a chunk of size 108, which is longer than the specified 100
Created a chunk of size 112, which is longer than the specified 100
Created a chunk of size 119, which is longer than the specified 100
Created a chunk of size 120, which is longer than the specified 100
Created a chunk of size 121, which is longer than the specified 100
Created a chunk of size 121, which is longer than the specified 100


In [36]:
print(type(char_chunks))
print(type(char_chunks[0]))
for i, chunk in enumerate(char_chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")   

<class 'list'>
<class 'str'>
Chunk 1:
Python is a widely-used high-level programming language celebrated for its clear syntax and ease of learning.

Chunk 2:
Developed by Guido van Rossum and first released in 1991, Python emphasizes readability and simplicity,

Chunk 3:
making it accessible to both beginners and seasoned developers. It supports multiple programming paradigms,

Chunk 4:
including procedural, object-oriented, and functional programming. Python's extensive standard library and vibrant

Chunk 5:
community contribute to its versatility, enabling it to be utilized across various domains such as web development,

Chunk 6:
data analysis, artificial intelligence, machine learning, automation, and scientific computing. Its adaptability and

Chunk 7:
continuous development ensure that Python remains a fundamental tool in the software development industry, fostering

Chunk 8:
innovation and productivity.



## Nice, but why we do not see any overlap - we asked for it?
The reason is, that we specified the new line character as separator.

Change to space:

In [43]:
char_splitter = CharacterTextSplitter(
    separator=" ",        # Split at space
    chunk_size=100,       # Each chunk will be at most 100 characters
    chunk_overlap=20,     # Overlap between chunks
    length_function=len    # Function to calculate length (default is len
)

char_chunks = char_splitter.split_text(text)
for i, chunk in enumerate(char_chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")

Chunk 1:
Python is a widely-used high-level programming language celebrated for its clear syntax and ease of

Chunk 2:
syntax and ease of learning. 
 Developed by Guido van Rossum and first released in 1991, Python

Chunk 3:
in 1991, Python emphasizes readability and simplicity, 
 making it accessible to both beginners and

Chunk 4:
both beginners and seasoned developers. It supports multiple programming paradigms, 
 including

Chunk 5:
including procedural, object-oriented, and functional programming. Python's extensive standard

Chunk 6:
extensive standard library and vibrant 
 community contribute to its versatility, enabling it to be

Chunk 7:
enabling it to be utilized across various domains such as web development, 
 data analysis,

Chunk 8:
data analysis, artificial intelligence, machine learning, automation, and scientific computing.

Chunk 9:
computing. Its adaptability and 
 continuous development ensure that Python remains a fundamental

Chunk 10:
a fundamental tool in the s

# Recursive Character Splitter

Here we use a list of possible separators.<br>
The __selection of the correct separators in the correct order__ is very important to get the imagined result!

In [44]:
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

In [45]:
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

rec_chunks = recursive_splitter.split_text(text)
for i, chunk in enumerate(rec_chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")

Chunk 1:
Python is a widely-used high-level programming language celebrated for its clear syntax and ease of

Chunk 2:
syntax and ease of learning.

Chunk 3:
Developed by Guido van Rossum and first released in 1991, Python emphasizes readability and

Chunk 4:
readability and simplicity,

Chunk 5:
making it accessible to both beginners and seasoned developers. It supports multiple

Chunk 6:
supports multiple programming paradigms,

Chunk 7:
including procedural, object-oriented, and functional programming. Python's extensive standard

Chunk 8:
extensive standard library and vibrant

Chunk 9:
community contribute to its versatility, enabling it to be utilized across various domains such

Chunk 10:
domains such as web development,

Chunk 11:
data analysis, artificial intelligence, machine learning, automation, and scientific computing.

Chunk 12:
computing. Its adaptability and

Chunk 13:
continuous development ensure that Python remains a fundamental tool in the software

Chunk 14:
in th

# Token Based Splitting

This one is not based on characters, but __tokens__, practically __words__ (character series separated by something white space).<br>
The size and overlap here does not count the characters, but the tokens.

In [46]:
from langchain_text_splitters.base import TokenTextSplitter

In [47]:
token_splitter = TokenTextSplitter(
    chunk_size=50,            # Each chunk will be at most 50 tokens    
    chunk_overlap=10          # Overlap between chunks
)

token_chunks = token_splitter.split_text(text)
for i, chunk in enumerate(token_chunks):    
    print(f"Chunk {i+1}:\n{chunk}\n")

Chunk 1:
Python is a widely-used high-level programming language celebrated for its clear syntax and ease of learning. 
    Developed by Guido van Rossum and first released in 1991, Python emphasizes readability and simplicity, 
 

Chunk 2:
 Python emphasizes readability and simplicity, 
    making it accessible to both beginners and seasoned developers. It supports multiple programming paradigms, 
    including procedural, object-oriented, and functional programming. Python's extensive standard

Chunk 3:
oriented, and functional programming. Python's extensive standard library and vibrant 
    community contribute to its versatility, enabling it to be utilized across various domains such as web development, 
    data analysis, artificial intelligence, machine learning

Chunk 4:
   data analysis, artificial intelligence, machine learning, automation, and scientific computing. Its adaptability and 
    continuous development ensure that Python remains a fundamental tool in the software 

# Pros and Cons

| | Character | Recursive | Token |
| -- | -- | -- | -- |
| __Pros__ | Simple and predictible | Respects text structure | Respects model token limits |
| | Good for structured text | Tries multiple separators | More accurate for embeddings |
| | | Best general-purpose splitter | |
| __Cons__ | May break mid-sentence | Slightly more complex | __Slower__ than character based |
| Use when: | Text has clear delimiters | Default choice for most texts | Working with token-limited models |